In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 62.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 110.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.6 MB/s eta 0:00:00


In [2]:
import pandas as pd
import torch
from torch.nn import CrossEntropyLoss
from torch.optim import Adam
from transformers import BertTokenizer, BertForSequenceClassification, BertConfig
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tqdm import tqdm

In [3]:
# 데이터 불러오기
df = pd.read_csv('complete.csv')
df['text'] = df['title'] + ' ' + df['content']
# NaN 값을 포함하는 행 삭제
df.dropna(subset=['text'], inplace=True)

In [4]:
# 원본 데이터를 학습 데이터와 테스트 데이터로 나눔
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [5]:
# KoBERT 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained('monologg/kobert')

In [6]:
def encode_data(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []

    for text in texts:
        encoded_dict = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=max_len,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        # 직접 패딩 수행
        padding_length = max_len - len(encoded_dict['input_ids'][0])
        all_tokens.append(torch.cat([encoded_dict['input_ids'], torch.zeros((1, padding_length), dtype=torch.long)], dim=1))
        all_masks.append(torch.cat([encoded_dict['attention_mask'], torch.zeros((1, padding_length), dtype=torch.long)], dim=1))

    all_tokens = torch.cat(all_tokens, dim=0)
    all_masks = torch.cat(all_masks, dim=0)

    return all_tokens, all_masks

In [7]:
# 학습 데이터 인코딩
train_input_ids, train_attention_masks = encode_data(train_df['text'], tokenizer)
train_labels1 = torch.tensor(train_df['label1'].values) - 1

# 테스트 데이터 인코딩 및 성능 평가
test_input_ids, test_attention_masks = encode_data(test_df['text'], tokenizer)
test_labels1 = torch.tensor(test_df['label1'].values) - 1

In [8]:
# 데이터셋 및 데이터 로더 생성
train_dataset = TensorDataset(train_input_ids, train_attention_masks, train_labels1)
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)

test_dataset = TensorDataset(test_input_ids, test_attention_masks, test_labels1)
test_dataloader = DataLoader(test_dataset, batch_size=2, shuffle=False)

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# 모델 정의
class KoBERTForSingleLabelClassification(torch.nn.Module):
    def __init__(self, num_labels):
        super(KoBERTForSingleLabelClassification, self).__init__()
        self.config = BertConfig.from_pretrained('monologg/kobert', num_labels=num_labels, output_hidden_states=True)
        self.bert = BertForSequenceClassification.from_pretrained('monologg/kobert', config=self.config)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        return outputs.logits


model = KoBERTForSingleLabelClassification(df['label1'].nunique()).to(device)
optimizer = Adam(model.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
test_inputs, val_inputs, test_masks, val_masks, test_labels1, val_labels1 = train_test_split(
    test_input_ids, test_attention_masks, test_labels1, random_state=42, test_size=0.5
)

test_dataset = TensorDataset(test_inputs, test_masks, test_labels1)
val_dataset = TensorDataset(val_inputs, val_masks, val_labels1)
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=False)
val_dataloader = DataLoader(val_dataset, batch_size=4, shuffle=True)

In [11]:
EPOCHS = 10
train_losses = []
best_accuracy = 0
best_model_path = "best_model_label1.pth"

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    total_correct = 0
    total_samples = 0

    for batch in tqdm(train_dataloader, desc=f"Training Epoch {epoch+1}/{EPOCHS}"):
        optimizer.zero_grad()
        input_ids, attention_mask, label = [b.to(device) for b in batch]
        logits = model(input_ids, attention_mask=attention_mask)

        loss = loss_fn(logits, label)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_correct += (logits.argmax(dim=1) == label).sum().item()
        total_samples += label.size(0)

    avg_train_loss = total_loss / total_samples
    train_losses.append(avg_train_loss)

    # Evaluate on validation set
    model.eval()
    val_correct = 0
    val_samples = 0

    with torch.no_grad():
        for batch in val_dataloader:
            input_ids, attention_mask, label = [b.to(device) for b in batch]
            logits = model(input_ids, attention_mask=attention_mask)

            val_correct += (logits.argmax(dim=1) == label).sum().item()
            val_samples += label.size(0)

    val_accuracy = val_correct / val_samples

    # 최고 정확도를 갱신할 경우 모델 저장
    if val_accuracy > best_accuracy:
        best_accuracy = val_accuracy
        torch.save(model.state_dict(), best_model_path)
        print(f"Model saved for epoch {epoch+1} with accuracy: {best_accuracy:.4f}")

Training Epoch 1/10: 100%|██████████| 500/500 [00:53<00:00,  9.32it/s]


Model saved for epoch 1 with accuracy: 0.4800


Training Epoch 3/10: 100%|██████████| 500/500 [00:50<00:00,  9.92it/s]


Model saved for epoch 3 with accuracy: 0.5080


Training Epoch 7/10: 100%|██████████| 500/500 [00:50<00:00,  9.87it/s]


Model saved for epoch 7 with accuracy: 0.5160


Training Epoch 8/10: 100%|██████████| 500/500 [00:50<00:00,  9.92it/s]


Model saved for epoch 8 with accuracy: 0.5440


Training Epoch 10/10: 100%|██████████| 500/500 [00:50<00:00,  9.92it/s]


Model saved for epoch 10 with accuracy: 0.5520


In [12]:
# 학습이 끝난 후 가장 좋은 성능을 보인 모델을 불러옵니다.
model.load_state_dict(torch.load(best_model_path))
model.eval()

# 테스트 데이터셋에 대한 예측을 수행
true_labels1 = []
predicted_labels1 = []

with torch.no_grad():
    for batch in test_dataloader:
        input_ids, attention_mask, label1 = [b.to(device) for b in batch]
        logits1 = model(input_ids, attention_mask)

        true_labels1.extend(label1.tolist())
        predicted_labels1.extend(logits1.argmax(dim=1).tolist())


# f1-score를 포함한 성능 지표를 출력
accuracy1 = sum([1 if true == pred else 0 for true, pred in zip(true_labels1, predicted_labels1)]) / len(true_labels1)

f1_1 = f1_score(true_labels1, predicted_labels1, average='weighted')


print(f"Test Accuracy for Label1: {accuracy1:.4f}")
print(f"F1 Score for Label1: {f1_1:.4f}")

Test Accuracy for Label1: 0.5360
F1 Score for Label1: 0.5015


In [13]:
print(len(train_labels1))
print(len(val_labels1))
print(len(test_labels1))

2000
250
250
